In [1]:
!pip install flwr tensorflow numpy pandas scikit-learn
!pip install -q flwr[simulation] tensorflow
!pip install ray[rllib]  # also recommended: ray[debug]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.2/512.2 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.68.1
    Uninstalling grpcio-1.68.1:
      Successfully uninstalled grpcio-1.68.1
  Attempting uninstall: cryptography
    Found existing installation: cryptography 43.0.3
    Uninstalling cryptography-43.0.3:
      Successfully uninstalled cryptography-43.0.3
  Attempting uninstall: typer
    Found existing installation: typer 0.14.0
    Uninstalling typer-0.14.0:
      Successfully uninstalled typer-0.14.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92

In [2]:
# Connect Driver
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# import libraries
import ray
from pyarrow import csv
import matplotlib.pyplot as plt
import seaborn as sns
import flwr as fl
from flwr.common import Metrics
from flwr.simulation.ray_transport.utils import enable_tf_gpu_growth
import math
from typing import Dict, List, Tuple
import glob
from flwr.simulation import run_simulation
import tensorflow as tf

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, precision_recall_curve, auc, average_precision_score, f1_score
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
import sys
import sklearn
import io
import random
import matplotlib.pyplot as plt
from keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D, GRU
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder, Normalizer
from sklearn.model_selection import train_test_split


#Features - UNSWNB-15

In [ ]:
CSV_HEADER = [
    "dur", "proto", "service", "state", "spkts", "dpkts", "sbytes", "dbytes", "rate", "sttl", "dttl",
    "sload", "dload", "sloss", "dloss", "sinpkt", "dinpkt", "sjit", "djit", "swin", "stcpb", "dtcpb",
    "dwin", "tcprtt", "synack", "ackdat", "smean", "dmean", "trans_depth", "response_body_len",
    "ct_srv_src", "ct_state_ttl", "ct_dst_ltm", "ct_src_dport_ltm", "ct_dst_sport_ltm",
    "ct_dst_src_ltm", "is_ftp_login", "ct_ftp_cmd", "ct_flw_http_mthd", "ct_src_ltm",
    "ct_srv_dst","is_sm_ips_ports","label"
] # drop attack_cat,
CSV_HEADER_WITHOUT_LABEL = [
    "dur", "proto", "service", "state", "spkts", "dpkts", "sbytes", "dbytes", "rate", "sttl", "dttl",
    "sload", "dload", "sloss", "dloss", "sinpkt", "dinpkt", "sjit", "djit", "swin", "stcpb", "dtcpb",
    "dwin", "tcprtt", "synack", "ackdat", "smean", "dmean", "trans_depth", "response_body_len",
    "ct_srv_src", "ct_state_ttl", "ct_dst_ltm", "ct_src_dport_ltm", "ct_dst_sport_ltm",
    "ct_dst_src_ltm", "is_ftp_login", "ct_ftp_cmd", "ct_flw_http_mthd", "ct_src_ltm",
    "ct_srv_dst","is_sm_ips_ports"
] # drop attack_cat,

#UNSWNB-15

In [ ]:
def read_data(path_train,path_test): #CSV_HEADER
  # train_data = pd.read_csv(path_train, header=None, names=CSV_HEADER)
  # test_data = pd.read_csv(path_test, header=None, names=CSV_HEADER)
  train_data = pd.read_csv(path_train)
  test_data = pd.read_csv(path_test)
  train_data = train_data.drop(['attack_cat'], axis=1)
  test_data = test_data.drop(['attack_cat'], axis=1)
  train_data = train_data.drop(['id'], axis=1)
  test_data = test_data.drop(['id'], axis=1)
  print(f"Train dataset shape: {train_data.shape}")
  print(f"Test dataset shape: {test_data.shape}")
  return train_data, test_data
#/content/drive/MyDrive/BigDataAgain/Dataset/NSL-KDD/NSL_KDD_Train.csv
#/content/drive/MyDrive/BigDataAgain/Dataset/NSL-KDD/NSL_KDD_Test.csv

In [ ]:
def preprocess_categorical_features(train_data,test_data):
  train_data['service']=train_data['service'].apply(lambda x: 'none' if x=='-' else x )
  test_data['service']=test_data['service'].apply(lambda x: 'none' if x=='-' else x )
  train_data['state']=train_data['state'].apply(lambda x: 'none' if x=='-' else x )
  test_data['state']=test_data['state'].apply(lambda x: 'none' if x=='-' else x )
  CATEGORICAL_FEATURE_NAMES = ["proto","service","state"]
  # Danh sách các cột cần Label Encode
  CATEGORICAL_FEATURE_NAMES = ["proto","service","state"]

  # Label Encoding cho từng cột
  label_encoders = {}
  for col in CATEGORICAL_FEATURE_NAMES:
      label_encoder = LabelEncoder()
      train_data[col] = label_encoder.fit_transform(train_data[col])
      test_data[col] = label_encoder.fit_transform(test_data[col])
      label_encoders[col] = label_encoder  # Lưu lại encoder nếu cần inverse_transform sau này
  return train_data,test_data

In [ ]:
def preprocess_numeric_features(train_data,test_data):
  #Preprocessing Numerical Features.
  scaler_train = StandardScaler()
  scaler_train = scaler_train.fit(train_data[CSV_HEADER_WITHOUT_LABEL])
  train_data[CSV_HEADER_WITHOUT_LABEL] = scaler_train.transform(train_data[CSV_HEADER_WITHOUT_LABEL])

  scaler_test = StandardScaler()
  scaler_test = scaler_test.fit(test_data[CSV_HEADER_WITHOUT_LABEL])
  test_data[CSV_HEADER_WITHOUT_LABEL] = scaler_test.transform(test_data[CSV_HEADER_WITHOUT_LABEL])
  return train_data,test_data

In [ ]:
def exchange_label(train_data,test_data):

  #Convert "label" into normal=0 and attack=1 for KDDTrain+
  train_data['label']=train_data['label'].apply(lambda x: 'ibnormal' if x==1 else 'normal')
  #train_data.drop(['label'], axis=1)

  #Convert "label" into normal=0 and attack=1 for KDDTest+
  test_data['label']=test_data['label'].apply(lambda x: 'ibnormal' if x==1 else 'normal')
  #test_data.drop(['label'],1)

  train_data = train_data.fillna(axis=0, method='ffill')
  test_data = test_data.fillna(axis=0,method='ffill')
  return train_data, test_data

In [ ]:
def save_file(train_data,test_data):
  train_data_file = "train_data.csv"
  test_data_file = "test_data.csv"

  train_data.to_csv(train_data_file, index=False, header=False)
  test_data.to_csv(test_data_file, index=False, header=False)
  return train_data_file, test_data_file

In [ ]:
train_data,test_data = read_data("/content/drive/MyDrive/AnToanHeThongThongTin-CH2024/UNSW_NB15_training-set.csv","/content/drive/MyDrive/AnToanHeThongThongTin-CH2024/UNSW_NB15_testing-set.csv")
train_data,test_data = preprocess_categorical_features(train_data,test_data)
train_data,test_data = preprocess_numeric_features(train_data,test_data)
#train_data, test_data = exchange_label(train_data,test_data)
train_data_1 ,train_data_2 = train_test_split(train_data,test_size=0.5)
train_data_1.to_csv("train_data_1.csv", index=False, header=True)
train_data_2.to_csv("train_data_2.csv", index=False, header=True)
test_data.to_csv("test_data.csv", index=False, header=True)
#train_data_file,test_data_file = save_file(train_data,test_data)

Train dataset shape: (175341, 43)
Test dataset shape: (82332, 43)


#Features - CIC_UNSW-NB15

In [4]:
CSV_HEADER=['Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
       'Total Length of Fwd Packet', 'Total Length of Bwd Packet',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Packet Length Min', 'Packet Length Max', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
       'URG Flag Count', 'CWR Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
       'Average Packet Size', 'Fwd Segment Size Avg', 'Bwd Segment Size Avg',
       'Fwd Bytes/Bulk Avg', 'Fwd Packet/Bulk Avg', 'Fwd Bulk Rate Avg',
       'Bwd Bytes/Bulk Avg', 'Bwd Packet/Bulk Avg', 'Bwd Bulk Rate Avg',
       'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Packets',
       'Subflow Bwd Bytes', 'FWD Init Win Bytes', 'Bwd Init Win Bytes',
       'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
       'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max',
       'Idle Min', 'Label', 'attack_cat'] # bo Src_IP, Dst_IP, Timestamp, {'Flow ID', 'Src Port', 'Dst Port', 'Protocol'}
CSV_HEADER_WITHOUT_LABEL = ['Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
       'Total Length of Fwd Packet', 'Total Length of Bwd Packet',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Packet Length Min', 'Packet Length Max', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
       'URG Flag Count', 'CWR Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
       'Average Packet Size', 'Fwd Segment Size Avg', 'Bwd Segment Size Avg',
       'Fwd Bytes/Bulk Avg', 'Fwd Packet/Bulk Avg', 'Fwd Bulk Rate Avg',
       'Bwd Bytes/Bulk Avg', 'Bwd Packet/Bulk Avg', 'Bwd Bulk Rate Avg',
       'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Packets',
       'Subflow Bwd Bytes', 'FWD Init Win Bytes', 'Bwd Init Win Bytes',
       'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
       'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max',
       'Idle Min']

# Bảng mã hóa
encoded_to_original = {
    0: 'Benign',
    1: 'Analysis',
    2: 'Backdoor',
    3: 'DoS',
    4: 'Exploits',
    5: 'Fuzzers',
    6: 'Generic',
    7: 'Reconnaissance',
    8: 'Shellcode',
    9: 'Worms'
}
# decoded_labels = [encoded_to_original[label] for label in y_pred]
# decoded_labels

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#CIC_UNSW_NB15

In [33]:
df = pd.read_csv("/content/CIC_UNSW-LABEL_Attack_cat.csv")
df = df.drop(['attack_cat'], axis=1)
scaler = StandardScaler()
scaler = scaler.fit(df[CSV_HEADER_WITHOUT_LABEL])
df[CSV_HEADER_WITHOUT_LABEL] = scaler.transform(df[CSV_HEADER_WITHOUT_LABEL])

In [35]:
#Chia data_test
train_data,test_data = train_test_split(df,test_size=0.2)
train_data_1 ,train_data_2 = train_test_split(train_data,test_size=0.5)
train_data_1.to_csv("train_data_1.csv", index=False, header=True)
train_data_2.to_csv("train_data_2.csv", index=False, header=True)
train_data.to_csv("train_data.csv", index=False, header=True)
test_data.to_csv("test_data.csv", index=False, header=True)

#Client and Server for Distributed

In [36]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, x_train, y_train,model) -> None:
        # Create model
        self.model = model
        self.x_train, self.y_train = x_train, y_train

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):

        self.model.set_weights(parameters)
        self.model.fit(
             self.x_train,  self.y_train, epochs=3, batch_size=32, verbose=VERBOSE
        )
        return self.model.get_weights(), len(self.x_train), {}

In [37]:
def get_client_fn(dataset_partitions,model):
    """Return a function to be executed by the VirtualClientEngine in order to construct
    a client."""

    def client_fn(cid: str) -> fl.client.Client:
        """Construct a FlowerClient with its own dataset partition."""

        # Extract partition for client with id = cid
        x_train, y_train = dataset_partitions[int(cid)]

        # Create and return client
        return FlowerClient(x_train, y_train, model)

    return client_fn


def partition_dataset(x, y, num_clients):
    partition_size = math.floor(len(x) / num_clients)
    partitions = []
    for i in range(num_clients):
        idx_from, idx_to = i * partition_size, (i + 1) * partition_size
        partitions.append((x[idx_from:idx_to], y[idx_from:idx_to]))
    return partitions

def get_evaluate_fn(model,file_path):
    """Return an evaluation function for server-side (i.e. centralized) evaluation."""

    # The `evaluate` function will be called after every round by the strategy
    def evaluate(
        server_round: int,
        parameters: fl.common.NDArrays,
        config: Dict[str, fl.common.Scalar],
    ):
        model.set_weights(parameters)  # Update model with the latest parameters
        model.save(file_path)
        # model_report(model,x_test,y_test)
        return

    return evaluate


In [38]:
def get_partitions(data_path):
    partitions = []

    for f in glob.glob(data_path):
        data = pd.read_csv(f, header=0)

        data.columns = data.columns.str.strip()

        # data = data[data["Class"] != ' null']
        # data["Class"] = data["Class"].replace({' "-1"' : 0 , ' "Facet"' : 1 })
        # data["Class"] = data["Class"].replace({'-1' : 0, 'Facet' : 1 })
        # data["Class"] = data["Class"].replace({' -1' : 0, ' Facet' : 1 })
        # data["Class"] = data["Class"].replace(np.nan,'benign')
        #data["label"] = np.where(data["label"]=='normal',0,1) # 'normal' -> 0, other ->1
        #data = data.sample(frac = 1).reset_index(drop=True)

        x_data = data.iloc[:,:-1]
        y_data = data["Label"]

        partitions.append((x_data.to_numpy(),y_data.to_numpy()))

    return partitions

In [ ]:
def model_report_list(model, df_test):
    result = []

    x_test = df_test.iloc[:, :-1]
    y_test = df_test["label"]

    # Get model predictions for the test data
    y_pred_prob = model.predict(x_test, verbose=0)
    y_pred = (y_pred_prob > 0.5).astype("int32")

    df_test["PLabel"] = y_pred

    # # Calculate overall accuracy
    # overall_accuracy = accuracy_score(y_test, y_pred)
    # result.append(('Overall Accuracy', overall_accuracy))

    # Calculate F1 Score
    f1 = f1_score(y_test, y_pred)
    result.append(('F1 Score', f1))

    # Calculate AUC
    auc_score = roc_auc_score(y_test, y_pred_prob)
    result.append(('AUC', auc_score))

    # # Calculate AUC-PR
    # precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)
    # auc_pr_score = auc(recall, precision)
    # result.append(('AUC-PR', auc_pr_score))

    # # Calculate and print per-class accuracy
    # unique_classes = np.unique(y_test)
    # confusion_mat = confusion_matrix(y_test, y_pred, labels=unique_classes)
    # per_class_accuracy = np.diag(confusion_mat) / np.sum(confusion_mat, axis=1)
    # for class_id, accuracy in zip(unique_classes, per_class_accuracy):
    #     result.append((f'Accuracy Class {class_id}', accuracy))

    # # Calculate False Positives and False Negatives
    # FP = confusion_mat[0][1]
    # FN = confusion_mat[1][0]
    # result.append(('False Positives', FP))
    # result.append(('False Negatives', FN))

    return result

#CREATE MODEL

In [77]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.neural_network import MLPClassifier

# ANN Model Creationg Function
def create_ann_model(input_shape=(42,), hidden_units=None):
    """
    Create a feedforward neural network model for binary classification with
    the specified input shape.

    Parameters:
    - input_shape: Tuple specifying the shape of the input data.
    - hidden_units: List of integers specifying the number of units in each hidden layer.
                    Default is [64, 32].

    Returns:
    - A compiled Keras model.
    """

    if hidden_units is None:
        hidden_units = [64, 32]

    model = Sequential()

    # Input layer
    model.add(Dense(hidden_units[0], activation='relu', input_shape=input_shape))
    model.add(Dropout(0.2))

    # Hidden layers
    for units in hidden_units[1:]:
        model.add(Dense(units, activation='relu'))
        model.add(Dropout(0.2))

    # Output layer for binary classification
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])
    return model

Xây dựng model cho phân loại

In [39]:
def create_ann_model_multiclass(input_shape=(76,), hidden_units=None, num_classes=10):
    """
    Create a feedforward neural network model for multiclass classification with
    the specified input shape and number of classes.

    Parameters:
    - input_shape: Tuple specifying the shape of the input data.
    - hidden_units: List of integers specifying the number of units in each hidden layer.
                    Default is [64, 32].
    - num_classes: Integer specifying the number of output classes (e.g., 10 for multiclass classification).

    Returns:
    - A compiled Keras model.
    """
    if hidden_units is None:
        hidden_units = [64, 32]

    model = Sequential()

    # Input layer
    model.add(Dense(hidden_units[0], activation='relu', input_shape=input_shape))
    model.add(Dropout(0.2))

    # Hidden layers
    for units in hidden_units[1:]:
        model.add(Dense(units, activation='relu'))
        model.add(Dropout(0.2))

    # Output layer for multiclass classification
    model.add(Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(
        loss='sparse_categorical_crossentropy',  # Loss function for integer labels
        optimizer='nadam',
        metrics=['accuracy']
    )

    return model

#Train and TEST_model

## Train model without Distributed

In [ ]:
# Đọc dữ liệu từ file CSV không có header
train_data = pd.read_csv('/content/train_data.csv', header=None)
  # Lấy tất cả các cột trừ cột cuối (label)
train_labels = train_data.values[:, -1]  # Lấy cột cuối làm nhãn (label)
train_labels = np.where(train_labels == 'normal', 0, 1) # 'normal' -> 0, other ->1
train_data = train_data.values[:, :-1].astype(np.float32)
truclb_model = create_ann_model()
truclb_model.fit(
    train_data,         # Dữ liệu đầu vào
    train_labels,       # Nhãn đầu ra
    epochs=1,          # Số epoch (lần lặp qua dữ liệu)
    batch_size=32,      # Kích thước batch
    validation_split=0.5  # Chia 20% dữ liệu để validation
)
truclb_model.save('truclb_model_ver1-0.keras')  # Lưu mô hình dưới dạng SavedModel

#Test model Without Distributed

In [ ]:
# Đọc dữ liệu từ file CSV không có header
test_data = pd.read_csv('/content/test_data.csv', header=None)
  # Lấy tất cả các cột trừ cột cuối (label)
test_labels = test_data.values[:, -1]  # Lấy cột cuối làm nhãn (label)
test_labels = np.where(test_labels == 'normal', 0, 1) # 'normal' -> 0, other ->1
test_data = test_data.values[:, :-1].astype(np.float32)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report,ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix , classification_report, recall_score, precision_score,f1_score
from tensorflow.keras.models import load_model
import tensorflow.keras.backend as K

model_reload = load_model('truclb_model_ver1-0.keras')
y_pred = model_reload.predict(test_data)
def print_Report(y_true,y_pred_bool):
    print(classification_report(y_true, y_pred_bool))
    #20/12
    #Confusion matrix -> vẽ và phân tích.
    print('_______________________CONFUSION MATRIX_______________________')
    cm = confusion_matrix(y_true,y_pred_bool)
    cmp = ConfusionMatrixDisplay(cm, display_labels=[0, 1])
    cmp.plot()
    return
print_Report(test_labels,K.round(y_pred))

#Train model with Distributed Learning

###Global setting


In [40]:
VERBOSE = 0
NUM_CLIENTS = 2

# With a dictionary, you tell Flower's VirtualClientEngine that each
# client needs exclusive access to these many resources in order to run
client_resources = {"num_cpus":2,"num_gpus": 0}

### Train

In [ ]:
class MedianFedAvg(fl.server.strategy.FedAvg):
    def aggregate_fit(self, rnd, results, failures):
        """
        Ghi đè phương thức aggregate_fit để sử dụng Median-based Federated Averaging.
        """
        if not results:
            return None, {}

        # Lấy trọng số mô hình từ tất cả các client
        weights = [fit_res.parameters.tensors for _, fit_res in results]

        # Chuyển đổi trọng số từ list sang numpy arrays
        client_weights = [[np.array(layer) for layer in model_weights] for model_weights in weights]

        # Tính trung vị cho từng lớp trọng số
        aggregated_weights = []
        for weights_per_layer in zip(*client_weights):  # Lặp qua từng lớp
            weight_array = np.array(weights_per_layer)
            median_weights = np.median(weight_array, axis=0)
            aggregated_weights.append(median_weights)

        # Trả về trọng số được tổng hợp dưới dạng `Parameters`
        aggregated_parameters = fl.common.ndarrays_to_parameters(aggregated_weights)
        return aggregated_parameters, {}

In [82]:
from flwr.simulation import run_simulation,start_simulation
result = []

partitions = get_partitions("./dataset/*")

for _ in range(2):

    hidden_units = [64,32]
    output_model = f"./models/i_medianFedAvg_MLP{_}.keras"

    #model = create_ann_model(hidden_units=hidden_units) #làm cho binary
    model = create_ann_model_multiclass(hidden_units=hidden_units) #Làm cho Multiclass
    #model = create_MLPClassifier()

    strategy = fl.server.strategy.FedAvg(
    fraction_fit=1,
    # min_fit_clients=10,
    fraction_evaluate = 1,
    min_available_clients = NUM_CLIENTS,
    evaluate_fn = get_evaluate_fn(model,output_model),
    )
#     strategy = MedianFedAvg(
#     fraction_fit=1,
#     fraction_evaluate=1,
#     min_available_clients=NUM_CLIENTS,
#     evaluate_fn=get_evaluate_fn(model, output_model),
# )

    # Start simulation
    history = fl.simulation.start_simulation(
    client_fn=get_client_fn(partitions,model),
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=1),
    strategy=strategy,
    client_resources=client_resources,
    # actor_kwargs={
    # "on_actor_init_fn": enable_tf_gpu_growth  },
    )
    del history
    del strategy
    del model

    # model = load_model(output_model)
    # s = model_report_list(model,test_data.copy())

    # result.append(s)
    # del model


	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=1, no round_timeout
2024-12-06 14:00:18,924	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'memory': 7910431950.0, 'object_store_memory': 3955215974.0, 'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0, 'CPU': 2.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 2, 'num_gpus': 0}
INFO :      Flower VCE: Creating VirtualClientEngineActorP

RuntimeError: Simulation crashed.

(ClientAppActor pid=18958) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=18958) 
(ClientAppActor pid=18958)             This is a deprecated feature. It will be removed
(ClientAppActor pid=18958)             entirely in future versions of Flower.
(ClientAppActor pid=18958)         
(ClientAppActor pid=18958) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.


# Test model with Distributed Training

In [42]:
# Prepare Data for Testing - Vẫn con Test data còn lưu lại.
test_df= pd.read_csv("./test_data.csv")

(ClientAppActor pid=9033) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=9033) 
(ClientAppActor pid=9033)             This is a deprecated feature. It will be removed
(ClientAppActor pid=9033)             entirely in future versions of Flower.
(ClientAppActor pid=9033)         
(ClientAppActor pid=9033) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during the

## FOR BINARY

In [ ]:
result = []
for _ in range(2):

    output_model = f"./models/i_medianFedAvg_{_}.keras"
    print('--------------------Model thứ: ', _ )
    model = load_model(output_model)
    s = model_report_list(model, test_data.copy())
    #s = model_report_list_multiclass(model, test_data.copy())

    result.append(s)
    del model

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, precision_recall_curve, auc, average_precision_score, f1_score, precision_score, recall_score
def model_report_list(model, df_test):
    result = []

    x_test = df_test.iloc[:, :-1]
    y_test = df_test["Label"]

    # Get model predictions for the test data
    y_pred_prob = model.predict(x_test, verbose=0)

    optimal_threshold = 0.5

    y_pred = (y_pred_prob > optimal_threshold).astype("int32")

    df_test["PLabel"] = y_pred

    # Calculate F1 Score
    f1 = f1_score(y_test, y_pred)
    result.append(('F1 Score', f1))

    # Calculate AUC
    auc_score = roc_auc_score(y_test, y_pred_prob)
    result.append(('AUC', auc_score))

    # Calculate Average Precision
    avg_precision = average_precision_score(y_test, y_pred_prob)
    result.append(('Average Precision', avg_precision))

    # Calculate Precision
    precision = precision_score(y_test, y_pred)
    result.append(('Precision', precision))

    # Calculate Recall
    recall = recall_score(y_test, y_pred)
    result.append(('Recall', recall))

    confusion_mat = confusion_matrix(y_test, y_pred)

    FP = confusion_mat[0][1]
    FN = confusion_mat[1][0]

    TP = confusion_mat[1][1]
    TN = confusion_mat[0][0]

    FPR = FP / (FP + TN) if (FP + TN) > 0 else 0  # Avoid division by zero
    FNR = FN / (FN + TP) if (FN + TP) > 0 else 0  # Avoid division by zero

    result.append(('False Positive Rate', FPR))
    result.append(('False Negative Rate', FNR))

    del model

    return result


In [ ]:
data = {}

# Iterate through each tuple in the first list to use as keys and create lists for their values
for metric, value in result[0]:
    data[metric] = []

# Fill the lists with values from each corresponding position in the remaining lists
for res in result:
    for i, (metric, value) in enumerate(res):
        data[metric].append(value)

# Convert the dictionary into a DataFrame
result_df = pd.DataFrame(data)

print('MEAN_AVG')
print("-"*100)
print(result_df.mean(axis=0))
print("-"*100)

##MULTI_CLASSES

In [60]:
from sklearn.metrics import classification_report, confusion_matrix

def model_report_list_multiclass(model, df_test):
    result = []

    x_test = df_test.iloc[:, :-1]
    y_test = df_test["Label"]

    # Get model predictions for the test data
    y_pred_prob = model.predict(x_test, verbose=0)

    # Get predicted labels for multiclass
    y_pred = y_pred_prob.argmax(axis=1)

    df_test["PLabel"] = y_pred

    mapped_y_pred = [encoded_to_original[label] for label in y_pred]
    mapped_y_test = [encoded_to_original[label] for label in y_test]

    # Generate classification report
    class_report = classification_report(mapped_y_test, mapped_y_pred, output_dict=True)
    #print(class_report)
    result.append(('Classification Report', class_report))

    # # Calculate confusion matrix
    # confusion_mat = confusion_matrix(y_test, y_pred)
    # result.append(('Confusion Matrix', confusion_mat))

    del model

    return result


Report-Multiclass

In [72]:
result = []
for _ in range(2):

    output_model = f"./models/i_medianFedAvg_{_}.keras"
    print('--------------------Model thứ: ', _ )
    model = load_model(output_model)
    #s = model_report_list(model, test_data.copy())
    s = model_report_list_multiclass(model, test_data.copy())

    result.append(s)
    del model

--------------------Model thứ:  0


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'nadam', because it has 15 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMet

--------------------Model thứ:  1


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [74]:
import pandas as pd

# Tạo một DataFrame để lưu các giá trị trung bình
df_avg_classes = pd.DataFrame(columns=['precision', 'recall', 'f1-score', 'support', 'accuracy'])

# Lấy các class từ báo cáo đầu tiên
classes = list(result[0][0][1].keys())  # result[0][0][1] chứa dictionary của classification report

# Xóa các chỉ số tổng hợp (macro avg, weighted avg, accuracy, support)
classes = [cls for cls in classes if cls not in ['accuracy', 'macro avg', 'weighted avg']]

# Tính trung bình cho từng class và lấy support từ báo cáo đầu tiên
for cls in classes:
    precision_avg = 0
    recall_avg = 0
    f1_avg = 0
    # Lặp qua các báo cáo trong result để tính trung bình precision, recall, f1-score
    for report in result:
        precision_avg += report[0][1][cls]['precision']
        recall_avg += report[0][1][cls]['recall']
        f1_avg += report[0][1][cls]['f1-score']

    # Tính trung bình
    num_reports = len(result)

    # Lấy support từ báo cáo đầu tiên
    support = result[0][0][1][cls]['support']

    df_avg_classes.loc[cls] = [
        precision_avg / num_reports,
        recall_avg / num_reports,
        f1_avg / num_reports,
        support,  # Thêm support vào DataFrame
        None  # Sẽ tính accuracy sau
    ]

# Tính trung bình cho các chỉ số tổng hợp (macro avg và weighted avg)
for avg_type in ['macro avg', 'weighted avg']:
    precision_avg = 0
    recall_avg = 0
    f1_avg = 0
    for report in result:
        precision_avg += report[0][1][avg_type]['precision']
        recall_avg += report[0][1][avg_type]['recall']
        f1_avg += report[0][1][avg_type]['f1-score']

    # Sử dụng support từ 'macro avg' trong báo cáo đầu tiên
    support = result[0][0][1][avg_type]['support'] if avg_type in result[0][0][1] and 'support' in result[0][0][1][avg_type] else None

    df_avg_classes.loc[avg_type] = [
        precision_avg / num_reports,
        recall_avg / num_reports,
        f1_avg / num_reports,
        support,  # Thêm support nếu có
        None  # Sẽ tính accuracy sau
    ]

# Tính trung bình accuracy cho tất cả các mô hình
accuracy_avg = 0
for report in result:
    accuracy_avg += report[0][1]['accuracy']

df_avg_classes['accuracy'] = accuracy_avg / num_reports

# In kết quả trung bình
print(df_avg_classes)

                precision    recall  f1-score  support  accuracy
Analysis         0.000000  0.000000  0.000000     78.0   0.90616
Backdoor         0.000000  0.000000  0.000000     92.0   0.90616
Benign           0.999185  0.972774  0.985802  71842.0   0.90616
DoS              0.954545  0.006187  0.012235    889.0   0.90616
Exploits         0.695126  0.688016  0.690889   6058.0   0.90616
Fuzzers          0.474881  0.841162  0.606743   5940.0   0.90616
Generic          0.808716  0.091790  0.164782    877.0   0.90616
Reconnaissance   0.680531  0.606782  0.641178   3362.0   0.90616
Shellcode        0.000000  0.000000  0.000000    401.0   0.90616
Worms            0.000000  0.000000  0.000000     44.0   0.90616
macro avg        0.461298  0.320671  0.310163  89583.0   0.90616
weighted avg     0.922732  0.906160  0.903324  89583.0   0.90616
